# Simple Chat using OCI Generative AI

The notebook sets up a simple chat application using Oracle Cloud Infrastructure (OCI) Generative AI services. Here's an overview of the entire notebook:

1. Imports and Environment Setup:

    - Imports necessary libraries such as os, ads, oracledb, dotenv, and oci_genai.
    - Loads environment variables from a .env file using load_dotenv().

2. Embeddings Initialization:

    - Creates an instance of OCIGenAIEmbeddings using environment variables for model ID, service endpoint, and compartment ID.

3. Database Connection:

    - Establishes a connection to an Oracle database using oracledb.connect() with credentials retrieved from environment variables.

4. Vector Store Setup:

    - Defines the table name "DOCS" for storing vectors.
    - Initializes the OracleVS vector store with the database connection, embeddings, and table name.

5. Retriever Setup:

    - Converts the vector store into a retriever using vector_store.as_retriever().

6. Prompt Template Definition:

    - Defines a prompt template for the RAG model to use when generating answers.
    - The template instructs the assistant to use retrieved context fragments to answer questions.

7. Chat Model Initialization:

    - Creates an instance of ChatOCIGenAI with specified parameters, including model ID, service endpoint, compartment ID, and other model settings.

8. RAG Model Setup:

    - Combines the chat model and retriever into a RetrievalQA model using RetrievalQA.from_chain_type().
    - Uses the defined prompt template for generating answers.

9. Example Usage:

    - Demonstrates how to use the RAG model to generate responses to user inputs.
    - Sends a prompt to the model and prints the generated response.

In [1]:
import os
import ads
import oracledb
from dotenv import load_dotenv
import oci_genai
%load_ext autoreload
%autoreload 2
    


In [8]:

import os
import ads
import oracledb
from dotenv import load_dotenv
import oci_genai
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI
from langchain_community.embeddings.oci_generative_ai import OCIGenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import OracleVS
import oracledb

load_dotenv()

embeddings = OCIGenAIEmbeddings(
    model_id=os.getenv('CON_GEN_AI_EMB_MODEL_ID'),
    service_endpoint=os.getenv('CON_GEN_AI_SERVICE_ENDPOINT'),
    compartment_id=os.getenv('CON_GEN_AI_COMPARTMENT_ID'),
)


connection = oracledb.connect(
    user=os.getenv('CON_ADB_DEV_USER_NAME'), 
    password=os.getenv('CON_ADB_DEV_PASSWORD'), 
    dsn=os.getenv('CON_ADB_DEV_SERVICE_NAME')
)


#vector store table name
table_name = "DOCS"

vector_store = OracleVS(connection, embeddings, table_name)


retriever = vector_store.as_retriever(
    # search_kwargs={ 'k': 100} <- parameter to define the number of documents to retrieve
)

rag_prompt_template = """You are an assistant for question-answering tasks.
            Please use only the following retrieved context fragments to answer the question.
            If you don't know the answer, say you the cuestion is out of my context.
            Always use all available data.:
            {context}
            Question: {question}
            """

rag_prompt = PromptTemplate.from_template(rag_prompt_template)


llm = ChatOCIGenAI(
        model_id         = os.getenv("CON_GEN_AI_CHAT_MODEL_ID"),
        service_endpoint = os.getenv("CON_GEN_AI_SERVICE_ENDPOINT"),
        compartment_id   = os.getenv("CON_GEN_AI_COMPARTMENT_ID"),
        provider         = "meta",
        is_stream        = True,
        auth_type        = os.getenv("CON_GEN_AI_AUTH_TYPE"),
        model_kwargs     = {
            "max_tokens"        : 1024,
            "temperature"       : 0.6,
            "top_p"             : 0.7,
            "top_k"             : 20,
            "frequency_penalty" : 0
        }
    )

rag = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": rag_prompt}
)


In [9]:
print(rag.invoke("what RAG mean and what do I need to create one?"))

{'query': 'what RAG mean and what do I need to create one?', 'result': "RAG stands for Retrieval Augmented Generation. It is a type of AI model that combines the strengths of information retrieval and natural language generation to produce factually accurate and contextually relevant responses.\n\nTo create a RAG system, you need to follow these key steps:\n\n1. Data Collection: Collect relevant, domain-specific textual data from various external sources, such as PDFs, structured documents, or text files.\n2. Data Preprocessing: Preprocess the collected data to create manageable and meaningful chunks. This involves cleaning the text, removing noise, and normalizing the text format.\n3. Vectorization: Convert the preprocessed text into vector embeddings using transformer-based models like BERT or Sentence Transformers.\n4. Indexing: Store the vector embeddings in a vector database optimized for fast similarity-based retrieval.\n5. Retrieval: Use a retriever to convert user queries into 

In [4]:
print(rag.invoke("what RAG mean and what do I need to create one?"))

{'query': 'what RAG mean and what do I need to create one?', 'result': 'RAG stands for Retrieval Augmented Generation. It is a model that combines Large Language Models (LLMs) with a retrieval mechanism, allowing it to access external data sources in real-time. This makes it suitable for scenarios requiring up-to-date or frequently changing information.\n\nTo create a RAG model, you would need to integrate two core components of NLP: Information Retrieval (IR) and Natural Language Processing (NLP). Additionally, RAG requires complex infrastructure, including vector databases, to function effectively.'}


In [5]:
print(rag.invoke("Can you talk about peruvian food?"))

{'query': 'Can you talk about peruvian food?', 'result': 'The question is out of my context.'}
